<a href="https://colab.research.google.com/github/aiswarya-180/Bank-Customer-Churn-Prediction/blob/main/MRP_Feature_Selection_%2B_Oversampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost


In [ ]:
# mount google
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_1 = pd.read_csv('/content/drive/My Drive/MRP/after_EDA.csv')
df_1.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,45,1,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,0,49,0,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,0,51,1,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,0,40,0,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,0,40,1,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [ ]:
df_1.shape

(10127, 20)

In [ ]:
categorical_cols = df_1.select_dtypes(include=['object']).columns
categorical_cols

Index(['Education_Level', 'Marital_Status', 'Income_Category',
       'Card_Category'],
      dtype='object')

# Catboost with SMOTENC & Label Encoding



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFECV
from imblearn.over_sampling import SMOTENC
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

X_cat = df_1.drop("Attrition_Flag", axis=1)
y_cat = df_1["Attrition_Flag"]

# Train-test split
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(
    X_cat, y_cat, test_size=0.2, random_state=42, stratify=y_cat
)

# Label encode categorical columns on train and test with consistent encoders
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_train_cat[col] = le.fit_transform(X_train_cat[col].astype(str))
    X_test_cat[col] = le.transform(X_test_cat[col].astype(str))
    label_encoders[col] = le

# Get categorical column indices for SMOTENC and CatBoost
cat_col_indices = [X_train_cat.columns.get_loc(col) for col in categorical_cols]

# Convert to numpy for SMOTENC (it expects np.ndarray or sparse matrix)
X_train_np = X_train_cat.values

smote_nc = SMOTENC(categorical_features=cat_col_indices, random_state=42)
X_train_res, y_train_res = smote_nc.fit_resample(X_train_np, y_train_cat)

# After resampling, features are numpy arrays; keep track of feature names
feature_names = X_train_cat.columns.tolist()

# Setup StratifiedKFold for RFECV
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Initialize CatBoostClassifier with random_state for reproducibility
catboost_estimator = CatBoostClassifier(verbose=0, random_state=42, iterations=100)

rfecv_catboost = RFECV(
    estimator=catboost_estimator,
    step=1,
    cv=cv,
    scoring='recall',
    min_features_to_select=5,
    n_jobs=-1,
    verbose=1
)

# Fit RFECV (input X_train_res must be numpy array)
rfecv_catboost.fit(X_train_res, y_train_res)

# Selected features mask
selected_mask = rfecv_catboost.support_

# Get selected feature names
selected_features_catboost = [feature_names[i] for i, keep in enumerate(selected_mask) if keep]

print(f"Optimal number of features selected: {rfecv_catboost.n_features_}")
print(f"Best CV recall: {max(rfecv_catboost.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_catboost}")

# Transform train and test sets with RFECV selector
X_train_selected = rfecv_catboost.transform(X_train_res)
X_test_np = X_test_cat.values  # convert test set to numpy array
X_test_selected = rfecv_catboost.transform(X_test_np)

# Train final CatBoost model using selected features and categorical info
final_cat_model = CatBoostClassifier(
    verbose=0,
    random_state=42,
    iterations=1000
)

final_cat_model.fit(
    X_train_selected,
    y_train_res,
    cat_features=[i for i, keep in enumerate(selected_mask) if keep and i in cat_col_indices]
)

# Predict on test set
y_pred_cat = final_cat_model.predict(X_test_selected)
y_pred_proba_cat = final_cat_model.predict_proba(X_test_selected)[:, 1]

print("\n--- CATBOOST CLASSIFICATION REPORT ---")
print(classification_report(y_test_cat, y_pred_cat))

# Dictionary to store all results
all_results = {}

# Store results - note y_test_cat (not y_test)
all_results['CatBoost'] = {
    'features': rfecv_catboost.n_features_,
    'cv_recall': max(rfecv_catboost.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test_cat, y_pred_cat),
    'precision': precision_score(y_test_cat, y_pred_cat),
    'recall': recall_score(y_test_cat, y_pred_cat),
    'f1': f1_score(y_test_cat, y_pred_cat),
    'auc': roc_auc_score(y_test_cat, y_pred_proba_cat),
    'selected_features': selected_features_catboost
}

print("\n--- CATBOOST MODEL RESULTS ---")
print(f"Selected Features ({len(selected_features_catboost)}): {selected_features_catboost}\n")
print(classification_report(y_test_cat, y_pred_cat))


Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Optimal number of features selected: 15
Best CV recall: 0.9829
Selected features: ['Customer_Age', 'Gender', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']

--- CATBOOST CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      1701
           1       0.95      0.87      0.91       325

    accuracy                           0.97      2026
   macro avg       0.96      0.93      0.94      2026
weighted avg       0.97      0.97      0.97      2026


--- CATBOOST MODEL RESULTS ---
Selected Features (15): ['Customer_Age', 'Gender', 'Dependent_cou

In [ ]:
all_results

{'CatBoost': {'features': np.int64(15),
  'cv_recall': np.float64(0.9829383629831625),
  'accuracy': 0.9708785784797631,
  'precision': 0.9463087248322147,
  'recall': 0.8676923076923077,
  'f1': 0.9052969502407705,
  'auc': np.float64(0.9931443042554153),
  'selected_features': ['Customer_Age',
   'Gender',
   'Dependent_count',
   'Months_on_book',
   'Total_Relationship_Count',
   'Months_Inactive_12_mon',
   'Contacts_Count_12_mon',
   'Credit_Limit',
   'Total_Revolving_Bal',
   'Avg_Open_To_Buy',
   'Total_Amt_Chng_Q4_Q1',
   'Total_Trans_Amt',
   'Total_Trans_Ct',
   'Total_Ct_Chng_Q4_Q1',
   'Avg_Utilization_Ratio']}}

# One Hot Encoding

In [ ]:
# Apply one-hot encoding
df_encoded = pd.get_dummies(df_1, columns=categorical_cols, drop_first=True)

df_encoded.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,...,Marital_Status_Single,Marital_Status_Unknown,Income_Category_$40K - $60K,Income_Category_$60K - $80K,Income_Category_$80K - $120K,Income_Category_Less than $40K,Income_Category_Unknown,Card_Category_Gold,Card_Category_Platinum,Card_Category_Silver
0,0,45,1,3,39,5,1,3,12691.0,777,...,False,False,False,True,False,False,False,False,False,False
1,0,49,0,5,44,6,1,2,8256.0,864,...,True,False,False,False,False,True,False,False,False,False
2,0,51,1,3,36,4,1,0,3418.0,0,...,False,False,False,False,True,False,False,False,False,False
3,0,40,0,4,34,3,4,1,3313.0,2517,...,False,True,False,False,False,True,False,False,False,False
4,0,40,1,3,21,5,1,0,4716.0,0,...,False,False,False,True,False,False,False,False,False,False


# Seperate X & Y

In [ ]:
X = df_encoded.drop(columns=['Attrition_Flag'])
y = df_encoded['Attrition_Flag']

In [ ]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# OverSampling - **SMOTE**

In [ ]:
# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Convert to DataFrame if needed
if not isinstance(X_train_smote, pd.DataFrame):
    X_train_smote = pd.DataFrame(X_train_smote, columns=X.columns)

# Scaling

In [ ]:

# Prepare scaled data
scaler = StandardScaler()
X_train_smote_scaled_array = scaler.fit_transform(X_train_smote)
X_train_smote_scaled = pd.DataFrame(X_train_smote_scaled_array, columns=X_train_smote.columns)
X_test_scaled_array = scaler.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled_array, columns=X_test.columns)

# Feature Selection & Model

In [ ]:
# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Dictionary to store all results
all_results = {}

## Xgboost

In [ ]:
# ===== 2. XGBOOST =====
print("\n" + "="*80)
print("2. XGBOOST ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_xgb = RFECV(
    estimator=xgb.XGBClassifier(random_state=42, n_estimators=100, eval_metric='logloss'),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_xgb.fit(X_train_smote, y_train_smote)

selected_features_xgb = X_train_smote.columns[rfecv_xgb.support_].tolist()
print(f"Optimal features selected: {rfecv_xgb.n_features_}")
print(f"Best CV recall: {max(rfecv_xgb.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_xgb}")

# Train final model
X_train_xgb = rfecv_xgb.transform(X_train_smote)
X_test_xgb = rfecv_xgb.transform(X_test)
xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')
xgb_model.fit(X_train_xgb, y_train_smote)
y_pred_xgb = xgb_model.predict(X_test_xgb)
y_pred_proba_xgb = xgb_model.predict_proba(X_test_xgb)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_xgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_xgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_xgb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_xgb):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_xgb):.4f}")

print("\n--- XGBOOST CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_xgb))

all_results['XGBoost'] = {
    'features': rfecv_xgb.n_features_,
    'cv_recall': max(rfecv_xgb.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_xgb),
    'precision': precision_score(y_test, y_pred_xgb),
    'recall': recall_score(y_test, y_pred_xgb),
    'f1': f1_score(y_test, y_pred_xgb),
    'auc': roc_auc_score(y_test, y_pred_proba_xgb),
    'selected_features': selected_features_xgb
}



2. XGBOOST ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 28
Best CV recall: 0.9896
Selected features: ['Customer_Age', 'Gender', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Education_Level_Doctorate', 'Education_Level_Graduate', 'Education_Level_High School', 'Education_Level_Post-Graduate', 'Education_Level_Uneducated', 'Education_Level_Unknown', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$40K - $60K', 'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K', 'Income_Category_Less than $40K', 'Income_Category_Unknown', 'Card_Category_Platinum']

--- Final Model Performance ---
Accuracy: 0.9679
Precision: 0.9248
Recall: 0.8708
F1-Score: 0.8970
AUC-ROC: 0.9909

--- XGBOOST CLASSIFICATION REPORT ---
         

In [ ]:
all_results

{'XGBoost': {'features': np.int64(28),
  'cv_recall': np.float64(0.9895566593083149),
  'accuracy': 0.9679170779861797,
  'precision': 0.9248366013071896,
  'recall': 0.8707692307692307,
  'f1': 0.8969889064976229,
  'auc': np.float64(0.9908614841948176),
  'selected_features': ['Customer_Age',
   'Gender',
   'Total_Relationship_Count',
   'Months_Inactive_12_mon',
   'Contacts_Count_12_mon',
   'Credit_Limit',
   'Total_Revolving_Bal',
   'Avg_Open_To_Buy',
   'Total_Amt_Chng_Q4_Q1',
   'Total_Trans_Amt',
   'Total_Trans_Ct',
   'Total_Ct_Chng_Q4_Q1',
   'Avg_Utilization_Ratio',
   'Education_Level_Doctorate',
   'Education_Level_Graduate',
   'Education_Level_High School',
   'Education_Level_Post-Graduate',
   'Education_Level_Uneducated',
   'Education_Level_Unknown',
   'Marital_Status_Married',
   'Marital_Status_Single',
   'Marital_Status_Unknown',
   'Income_Category_$40K - $60K',
   'Income_Category_$60K - $80K',
   'Income_Category_$80K - $120K',
   'Income_Category_Less th

## LightGBM

In [ ]:
# ===== 3. LIGHTGBM =====
print("\n" + "="*80)
print("3. LIGHTGBM ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_lgb = RFECV(
    estimator=lgb.LGBMClassifier(random_state=42, n_estimators=100, verbose=-1),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_lgb.fit(X_train_smote, y_train_smote)

selected_features_lgb = X_train_smote.columns[rfecv_lgb.support_].tolist()
print(f"Optimal features selected: {rfecv_lgb.n_features_}")
print(f"Best CV recall: {max(rfecv_lgb.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_lgb}")

# Train final model
X_train_lgb = rfecv_lgb.transform(X_train_smote)
X_test_lgb = rfecv_lgb.transform(X_test)
lgb_model = lgb.LGBMClassifier(random_state=42, verbose=-1)
lgb_model.fit(X_train_lgb, y_train_smote)
y_pred_lgb = lgb_model.predict(X_test_lgb)
y_pred_proba_lgb = lgb_model.predict_proba(X_test_lgb)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lgb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lgb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lgb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lgb):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_lgb):.4f}")

print("\n--- LIGHTGBM CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_lgb))

all_results['LightGBM'] = {
    'features': rfecv_lgb.n_features_,
    'cv_recall': max(rfecv_lgb.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_lgb),
    'precision': precision_score(y_test, y_pred_lgb),
    'recall': recall_score(y_test, y_pred_lgb),
    'f1': f1_score(y_test, y_pred_lgb),
    'auc': roc_auc_score(y_test, y_pred_proba_lgb),
    'selected_features': selected_features_lgb
}



3. LIGHTGBM ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 20
Best CV recall: 0.9887
Selected features: ['Customer_Age', 'Gender', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Education_Level_High School', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$80K - $120K']

--- Final Model Performance ---
Accuracy: 0.9654
Precision: 0.8923
Recall: 0.8923
F1-Score: 0.8923
AUC-ROC: 0.9909

--- LIGHTGBM CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.98      0.98      0.98      1701
           1       0.89      0.89      0.89       325

    accuracy                           0.97      2026
   macro avg       0.94      0.94      0.94     

## RF

In [ ]:
# ===== 4. RANDOM FOREST =====
print("\n" + "="*80)
print("4. RANDOM FOREST ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_rf = RFECV(
    estimator=RandomForestClassifier(random_state=42, n_estimators=100),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_rf.fit(X_train_smote, y_train_smote)

selected_features_rf = X_train_smote.columns[rfecv_rf.support_].tolist()
print(f"Optimal features selected: {rfecv_rf.n_features_}")
print(f"Best CV recall: {max(rfecv_rf.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_rf}")

# Train final model
X_train_rf = rfecv_rf.transform(X_train_smote)
X_test_rf = rfecv_rf.transform(X_test)
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train_rf, y_train_smote)
y_pred_rf = rf_model.predict(X_test_rf)
y_pred_proba_rf = rf_model.predict_proba(X_test_rf)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_rf):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_rf):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_rf):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_rf):.4f}")

print("\n--- RANDOM FOREST CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_rf))

all_results['Random Forest'] = {
    'features': rfecv_rf.n_features_,
    'cv_recall': max(rfecv_rf.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'precision': precision_score(y_test, y_pred_rf),
    'recall': recall_score(y_test, y_pred_rf),
    'f1': f1_score(y_test, y_pred_rf),
    'auc': roc_auc_score(y_test, y_pred_proba_rf),
    'selected_features': selected_features_rf
}


4. RANDOM FOREST ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 17
Best CV recall: 0.9900
Selected features: ['Customer_Age', 'Gender', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Marital_Status_Married', 'Marital_Status_Single']

--- Final Model Performance ---
Accuracy: 0.9541
Precision: 0.8558
Recall: 0.8585
F1-Score: 0.8571
AUC-ROC: 0.9835

--- RANDOM FOREST CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1701
           1       0.86      0.86      0.86       325

    accuracy                           0.95      2026
   macro avg       0.91      0.92      0.91      2026
weighted avg       0.95      0.95      0.95      2026



## Gradient boost

In [ ]:
# ===== 5. GRADIENT BOOSTING =====
print("\n" + "="*80)
print("5. GRADIENT BOOSTING ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_gb = RFECV(
    estimator=GradientBoostingClassifier(random_state=42, n_estimators=100),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_gb.fit(X_train_smote, y_train_smote)

selected_features_gb = X_train_smote.columns[rfecv_gb.support_].tolist()
print(f"Optimal features selected: {rfecv_gb.n_features_}")
print(f"Best CV recall: {max(rfecv_gb.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_gb}")

# Train final model
X_train_gb = rfecv_gb.transform(X_train_smote)
X_test_gb = rfecv_gb.transform(X_test)
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train_gb, y_train_smote)
y_pred_gb = gb_model.predict(X_test_gb)
y_pred_proba_gb = gb_model.predict_proba(X_test_gb)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_gb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_gb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_gb):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_gb):.4f}")

print("\n--- GRADIENT BOOSTING CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_gb))

all_results['Gradient Boosting'] = {
    'features': rfecv_gb.n_features_,
    'cv_recall': max(rfecv_gb.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_gb),
    'precision': precision_score(y_test, y_pred_gb),
    'recall': recall_score(y_test, y_pred_gb),
    'f1': f1_score(y_test, y_pred_gb),
    'auc': roc_auc_score(y_test, y_pred_proba_gb),
    'selected_features': selected_features_gb
}



5. GRADIENT BOOSTING ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 17
Best CV recall: 0.9721
Selected features: ['Customer_Age', 'Gender', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$80K - $120K', 'Card_Category_Platinum']

--- Final Model Performance ---
Accuracy: 0.9556
Precision: 0.8446
Recall: 0.8862
F1-Score: 0.8649
AUC-ROC: 0.9860

--- GRADIENT BOOSTING CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1701
           1       0.84      0.89      0.86       325

    accuracy                           0.96      2026
   macro avg       0.91      0.93      0.92      2026
weighted avg       0.96      0.96      0.

## Logistic Regression

In [ ]:
# ===== 6. LOGISTIC REGRESSION =====
print("\n" + "="*80)
print("6. LOGISTIC REGRESSION ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_lr = RFECV(
    estimator=LogisticRegression(random_state=42, max_iter=1000),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_lr.fit(X_train_smote_scaled, y_train_smote)

selected_features_lr = X_train_smote_scaled.columns[rfecv_lr.support_].tolist()
print(f"Optimal features selected: {rfecv_lr.n_features_}")
print(f"Best CV recall: {max(rfecv_lr.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_lr}")

# Train final model
X_train_lr = rfecv_lr.transform(X_train_smote_scaled)
X_test_lr = rfecv_lr.transform(X_test_scaled)
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train_lr, y_train_smote)
y_pred_lr = lr_model.predict(X_test_lr)
y_pred_proba_lr = lr_model.predict_proba(X_test_lr)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_lr):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_lr):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_lr):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_lr):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_lr):.4f}")

print("\n--- LOGISTIC REGRESSION CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_lr))

all_results['Logistic Regression'] = {
    'features': rfecv_lr.n_features_,
    'cv_recall': max(rfecv_lr.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_lr),
    'precision': precision_score(y_test, y_pred_lr),
    'recall': recall_score(y_test, y_pred_lr),
    'f1': f1_score(y_test, y_pred_lr),
    'auc': roc_auc_score(y_test, y_pred_proba_lr),
    'selected_features': selected_features_lr
}



6. LOGISTIC REGRESSION ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 26
Best CV recall: 0.9119
Selected features: ['Gender', 'Dependent_count', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Education_Level_Doctorate', 'Education_Level_Graduate', 'Education_Level_High School', 'Education_Level_Post-Graduate', 'Education_Level_Uneducated', 'Education_Level_Unknown', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$40K - $60K', 'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K', 'Income_Category_Less than $40K', 'Income_Category_Unknown']

--- Final Model Performance ---
Accuracy: 0.8870
Precision: 0.6548
Recall: 0.6246
F1-Score: 0.6394
AUC-ROC: 0.8829

--- LOGISTIC REGRESSION CLASSIFICATION REPORT ---
              precision 

## SVM

In [ ]:
# ===== 7. SVM =====
print("\n" + "="*80)
print("7. SVM ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_svm = RFECV(
    estimator=SVC(kernel='linear', random_state=42, probability=True),
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_svm.fit(X_train_smote_scaled, y_train_smote)

selected_features_svm = X_train_smote_scaled.columns[rfecv_svm.support_].tolist()
print(f"Optimal features selected: {rfecv_svm.n_features_}")
print(f"Best CV recall: {max(rfecv_svm.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_svm}")

# Train final model
X_train_svm = rfecv_svm.transform(X_train_smote_scaled)
X_test_svm = rfecv_svm.transform(X_test_scaled)
svm_model = SVC(kernel='linear', random_state=42, probability=True)
svm_model.fit(X_train_svm, y_train_smote)
y_pred_svm = svm_model.predict(X_test_svm)
y_pred_proba_svm = svm_model.predict_proba(X_test_svm)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_svm):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_svm):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_svm):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_svm):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_svm):.4f}")

print("\n--- SVM CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_svm))

all_results['SVM'] = {
    'features': rfecv_svm.n_features_,
    'cv_recall': max(rfecv_svm.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_svm),
    'precision': precision_score(y_test, y_pred_svm),
    'recall': recall_score(y_test, y_pred_svm),
    'f1': f1_score(y_test, y_pred_svm),
    'auc': roc_auc_score(y_test, y_pred_proba_svm),
    'selected_features': selected_features_svm
}



7. SVM ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 25
Best CV recall: 0.9066
Selected features: ['Gender', 'Dependent_count', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Education_Level_Doctorate', 'Education_Level_Graduate', 'Education_Level_High School', 'Education_Level_Post-Graduate', 'Education_Level_Uneducated', 'Education_Level_Unknown', 'Marital_Status_Married', 'Marital_Status_Single', 'Marital_Status_Unknown', 'Income_Category_$40K - $60K', 'Income_Category_$60K - $80K', 'Income_Category_$80K - $120K', 'Income_Category_Less than $40K', 'Income_Category_Unknown']

--- Final Model Performance ---
Accuracy: 0.8899
Precision: 0.6723
Recall: 0.6123
F1-Score: 0.6409
AUC-ROC: 0.8794

--- SVM CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.93   

## Naive Bayes

In [ ]:
def gaussian_nb_importance(estimator):
    """
    Custom importance getter for GaussianNB.
    Uses the absolute difference between class means as importance measure.
    """
    if not hasattr(estimator, 'theta_'):
        raise ValueError('Estimator must be fitted before calling importance getter')

    class_means = estimator.theta_  # Shape: (n_classes, n_features)
    if class_means.shape[0] == 2:  # Binary classification - FIXED
        importance = np.abs(class_means[0] - class_means[1])  # CORRECTED LINE
    else:  # Multi-class
        importance = np.var(class_means, axis=0)

    return importance

In [ ]:
# ===== 8. NAIVE BAYES  =====
print("\n" + "="*80)
print("8. NAIVE BAYES ALGORITHM")
print("="*80)

print("\n--- RFECV Feature Selection ---")
rfecv_nb = RFECV(
    estimator=GaussianNB(),
    importance_getter=gaussian_nb_importance,  # Custom importance getter
    step=1, cv=cv, scoring='recall', min_features_to_select=5, n_jobs=-1
)
rfecv_nb.fit(X_train_smote_scaled, y_train_smote)

selected_features_nb = X_train_smote_scaled.columns[rfecv_nb.support_].tolist()
print(f"Optimal features selected: {rfecv_nb.n_features_}")
print(f"Best CV recall: {max(rfecv_nb.cv_results_['mean_test_score']):.4f}")
print(f"Selected features: {selected_features_nb}")

# Train final model
X_train_nb = rfecv_nb.transform(X_train_smote_scaled)
X_test_nb = rfecv_nb.transform(X_test_scaled)
nb_model = GaussianNB()
nb_model.fit(X_train_nb, y_train_smote)
y_pred_nb = nb_model.predict(X_test_nb)
y_pred_proba_nb = nb_model.predict_proba(X_test_nb)[:, 1]

print("\n--- Final Model Performance ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred_nb):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_nb):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_nb):.4f}")
print(f"F1-Score: {f1_score(y_test, y_pred_nb):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba_nb):.4f}")

print("\n--- NAIVE BAYES CLASSIFICATION REPORT ---")
print(classification_report(y_test, y_pred_nb))

all_results['Naive Bayes'] = {
    'features': rfecv_nb.n_features_,
    'cv_recall': max(rfecv_nb.cv_results_['mean_test_score']),
    'accuracy': accuracy_score(y_test, y_pred_nb),
    'precision': precision_score(y_test, y_pred_nb),
    'recall': recall_score(y_test, y_pred_nb),
    'f1': f1_score(y_test, y_pred_nb),
    'auc': roc_auc_score(y_test, y_pred_proba_nb),
    'selected_features': selected_features_nb
}



8. NAIVE BAYES ALGORITHM

--- RFECV Feature Selection ---
Optimal features selected: 5
Best CV recall: 0.8507
Selected features: ['Total_Relationship_Count', 'Total_Revolving_Bal', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1']

--- Final Model Performance ---
Accuracy: 0.7828
Precision: 0.4083
Recall: 0.7877
F1-Score: 0.5378
AUC-ROC: 0.8431

--- NAIVE BAYES CLASSIFICATION REPORT ---
              precision    recall  f1-score   support

           0       0.95      0.78      0.86      1701
           1       0.41      0.79      0.54       325

    accuracy                           0.78      2026
   macro avg       0.68      0.78      0.70      2026
weighted avg       0.86      0.78      0.81      2026



In [ ]:
all_results

{'XGBoost': {'features': np.int64(28),
  'cv_recall': np.float64(0.9895566593083149),
  'accuracy': 0.9679170779861797,
  'precision': 0.9248366013071896,
  'recall': 0.8707692307692307,
  'f1': 0.8969889064976229,
  'auc': np.float64(0.9908614841948176),
  'selected_features': ['Customer_Age',
   'Gender',
   'Total_Relationship_Count',
   'Months_Inactive_12_mon',
   'Contacts_Count_12_mon',
   'Credit_Limit',
   'Total_Revolving_Bal',
   'Avg_Open_To_Buy',
   'Total_Amt_Chng_Q4_Q1',
   'Total_Trans_Amt',
   'Total_Trans_Ct',
   'Total_Ct_Chng_Q4_Q1',
   'Avg_Utilization_Ratio',
   'Education_Level_Doctorate',
   'Education_Level_Graduate',
   'Education_Level_High School',
   'Education_Level_Post-Graduate',
   'Education_Level_Uneducated',
   'Education_Level_Unknown',
   'Marital_Status_Married',
   'Marital_Status_Single',
   'Marital_Status_Unknown',
   'Income_Category_$40K - $60K',
   'Income_Category_$60K - $80K',
   'Income_Category_$80K - $120K',
   'Income_Category_Less th

# Deep Learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_selection import RFECV
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
import numpy as np

print("Deep Learning with RFECV Feature Selection")
print("="*50)

def create_simple_nn(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

class KerasWrapper(BaseEstimator, ClassifierMixin):
    def __init__(self, epochs=15, batch_size=32, verbose=0):
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None
        self.feature_importances_ = None

    def fit(self, X, y):
        y = np.array(y).reshape(-1, 1)
        self.model = create_simple_nn(X.shape[1])
        early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        self.model.fit(
            X, y,
            epochs=self.epochs,
            batch_size=self.batch_size,
            verbose=self.verbose,
            validation_split=0.2,
            callbacks=[early_stop]
        )
        self._calculate_feature_importance(X, y)
        return self

    def _calculate_feature_importance(self, X, y):
        baseline_pred = self.model.predict(X, verbose=0)
        baseline_loss = tf.keras.losses.binary_crossentropy(y, baseline_pred).numpy().mean()
        importances = []
        for i in range(X.shape[1]):
            X_permuted = X.copy()
            np.random.shuffle(X_permuted[:, i])
            permuted_pred = self.model.predict(X_permuted, verbose=0)
            permuted_loss = tf.keras.losses.binary_crossentropy(y, permuted_pred).numpy().mean()
            importance = max(0, permuted_loss - baseline_loss)
            importances.append(importance)
        self.feature_importances_ = np.array(importances)

    def predict(self, X):
        return (self.model.predict(X, verbose=0) > 0.5).astype(int).flatten()

# Instantiate wrapper
nn_estimator = KerasWrapper(epochs=15, verbose=0)

# Setup RFECV
rfecv = RFECV(
    estimator=nn_estimator,
    step=1,
    cv=StratifiedKFold(3),
    scoring='recall',
    min_features_to_select=5,
    n_jobs=1
)

print("Starting RFECV feature selection with SMOTE data...")
rfecv.fit(X_train_smote_scaled, y_train_smote)

# Selected features
if hasattr(X_train_smote_scaled, 'columns'):
    feature_names = X_train_smote_scaled.columns
else:
    feature_names = [f'Feature_{i}' for i in range(X_train_smote_scaled.shape[1])]

selected_features = [feature_names[i] for i, x in enumerate(rfecv.support_) if x]

print("\nSelected Features by RFECV:")
for i, feat in enumerate(selected_features, 1):
    print(f"{i}. {feat}")

print(f"\nOptimal number of features: {rfecv.n_features_}")
print(f"Best CV recall score: {max(rfecv.cv_results_['mean_test_score']):.4f}")

# Transform training and test data to selected features
X_train_selected = rfecv.transform(X_train_smote_scaled)
X_test_selected = rfecv.transform(X_test_scaled)

# Train final model on selected features
final_model = create_simple_nn(X_train_selected.shape[1])

final_model.fit(
    X_train_selected,
    np.array(y_train_smote).reshape(-1, 1),
    validation_data=(X_test_selected, np.array(y_test).reshape(-1, 1)),
    epochs=50,
    batch_size=32,
    verbose=1
)

# Predict and evaluate
y_pred = (final_model.predict(X_test_selected) > 0.5).astype(int).flatten()
y_pred_proba = final_model.predict(X_test_selected).flatten()

print("\nFinal Model Performance:")
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1-score: {f1_score(y_test, y_pred):.4f}")
print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")


Deep Learning with RFECV Feature Selection
Starting RFECV feature selection with SMOTE data...

Selected Features by RFECV:
1. Customer_Age
2. Total_Relationship_Count
3. Months_Inactive_12_mon
4. Total_Revolving_Bal
5. Total_Amt_Chng_Q4_Q1
6. Total_Trans_Amt
7. Total_Trans_Ct
8. Total_Ct_Chng_Q4_Q1
9. Education_Level_Doctorate
10. Education_Level_Graduate
11. Education_Level_High School
12. Education_Level_Post-Graduate
13. Education_Level_Uneducated
14. Education_Level_Unknown
15. Marital_Status_Married
16. Marital_Status_Single
17. Marital_Status_Unknown
18. Income_Category_$40K - $60K
19. Income_Category_$60K - $80K
20. Income_Category_$80K - $120K
21. Income_Category_Less than $40K
22. Income_Category_Unknown

Optimal number of features: 22
Best CV recall score: 0.8766
Epoch 1/50
425/425 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7676 - loss: 0.4680 - val_accuracy: 0.8885 - val_loss: 0.2925
Epoch 2/50
425/425 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9151 - loss: 0.2174 -